In [5]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

print("P1")
print("")

df1 = pd.read_csv('/Volumes/IMVDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['std_sign', 'glyph_img'])
df2 = pd.read_csv('/Volumes/IMVDrive/cfdb-django/glyph_count.csv', usecols=['std_sign', 'glyph']).sort_values(by='glyph')

start_file = '/Volumes/IMVDrive/cfdb-django/media/start_file.csv'

if 'start_file.csv' in os.listdir('/Volumes/IMVDrive/cfdb-django/media/'):
    os.remove(start_file)

f=open(start_file, 'a+')

df_group = df1.groupby(by=['std_sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

print("Thresholding number of glyphs per sign at 50...")
for name,group in df_group:
    if  group.count()[0] >= 50:
        if(os.stat(start_file).st_size == 0):
            group.reset_index().to_csv(f)
        else:
            group.reset_index().to_csv(f, header=None)
print("Thresholding finished.")

try:
    df_usable = pd.read_csv(start_file, usecols=['std_sign', 'glyph_img'], engine='python')
    df_usable.to_csv('/Volumes/IMVDrive/cfdb-django/media/start_file.csv')
except Exception:
    pass


print("")
print("P1")

P1

Thresholding number of glyphs per sign at 50...
Thresholding finished.

P1


In [6]:
from random import shuffle
import glob
import sys
import numpy as np
from PIL import Image

## ------------------------------------------------------------
## FORMAT CUTOM IMAGE DATA SET
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------------------------------------
## I use PIL instead of cv2
## ------------------------------------------------------------

print("FORMAT CUSTOM IMAGE INPUT")
print("")

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.trainBytesList(value=[value]))

'''
def load_image(addr):
    img = cv2.imread(addr)
    if img is None:
        return None 
    img = cv2.resize(img,(224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img
'''

def createDataRecord(outFileName, addrs, labels):
    print('Creating '+outFileName+'...')
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(outFileName)
    for i in range(len(addrs)):
        # print how many images are saved every 1000 images
        if not i % 1000:
            print(outFileName+' data: {}/{}'.format(i,len(addrs)))
            sys.stdout.flush()
        # load the image
        ## img = cv2.imread(glyphtosignlist.iloc[i,0])
        filename = os.fsdecode('/Volumes/imvDrive/cfdb-django/media/glyph_img/'+addrs.iloc[i])
        try:
            img = Image.open(filename).convert('L') #convert to grayscale
            
            img = img.resize((331,331)) ###uniform resize
            label = labels.iloc[i]
    
            # create a feature
            feature = {
                'glyph_img_raw': _bytes_feature(img.tostring()),
                'label_raw': _int64_feature(label)
            }
            
             # create an example protocol buffer
            example = tf.train.Example(features=tf.train.Features(feature=feature))

            # serialize to string and write on the file
            writer.write(example.SerializeToString())
            
        except Exception:
            pass
    print("")
        
    writer.close()
    sys.stdout.flush()
    
glyphtosignpath = pd.read_csv('/Volumes/imvDrive/cfdb-django/media/start_file.csv')
# read addresses and labels from the 'train' folder
# print(str(glyphtosignpath.iloc[:,1]))
addrs = glyphtosignpath.iloc[:,1]
labels = glyphtosignpath.iloc[:,2]
    
# divide the data into 80% train, 10% validation, and 10% test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
val_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]
  
createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)

print("DONE")

FORMAT CUSTOM IMAGE INPUT

Creating train.tfrecords...
train.tfrecords data: 0/1578
train.tfrecords data: 1000/1578

Creating val.tfrecords...
val.tfrecords data: 0/197

Creating test.tfrecords...
test.tfrecords data: 0/198

DONE


In [25]:
## ------------------------------
## READ INPUT: version 1
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------


import tensorflow as tf
import sys
import numpy as np
import matplotlib.pyplot as plt

channels = 1
batch_size = 10

def parser(record):
    keys_to_features = {
        "glyph_img_raw": tf.FixedLenFeature([], tf.string),
        "label_raw": tf.FixedLenFeature([], tf.int64)
    }
    
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["glyph_img_raw"], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, shape=[331, 331, channels])
    label = tf.cast(parsed["label_raw"], tf.int32)
    
    return image, label

def input_fn(filenames, train, batch_size=batch_size):
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    dataset = dataset.map(parser)
    
    '''
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
        num_repeat = None
    else:
        num_repeat = 1
        
    dataset = dataset.repeat(num_repeat)
    
    '''
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    images_batch, labels_batch = iterator.get_next()
    x = images_batch
    y = labels_batch
    
    return x, y

def train_input_fn():
    return input_fn(filenames=["train.tfrecords"], train=True)

def val_input_fn():
    return input_fn(filenames=["val.tfrecords"], train=False)

def test_input_fn():
    return input_fn(filenames=["test.tfrecords"], train=True)


## ------------------------------
## CONVOLUTIONAL NEURAL NETWORK
## Adapted from:
## https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
## https://www.tensorflow.org/tutorials/images/deep_cnn
## ------------------------------
    
def fc_relu(layer):
    W3 = tf.Variable(tf.truncated_normal([275560], stddev=0.05))
    flat_W3 = flat_layer(W3)
    B = tf.Variable(tf.constant(0.03,shape=[int(layer.shape[1])]))
    std_hypothesis = tf.matmul(layer, flat_W3) + B
    fc_relu = tf.nn.relu(std_hypothesis)
    return fc_relu

def flat_layer(layer):                  
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    return layer 

with tf.Session() as sess:
    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)

    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    img, lbl = train_input_fn()
    
    W1 = tf.Variable(tf.zeros([10,10,channels,channels], dtype=tf.float32)) 
    W2 = tf.Variable(tf.truncated_normal([5,5,channels,batch_size], stddev=0.05)) #https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
    
    conv1 = tf.nn.conv2d(img, W1, strides=[1, 1, 1, 1], padding="SAME") #stides=[batch_stride x_stride y_stride depth_stride]
    print("convolution 1.")
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    print("max pool 1.")
    norm1 = tf.nn.local_response_normalization(pool1,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    print("normalization 1.")
    
    print("")
    conv2 = tf.nn.conv2d(img, W2, strides=[1, 1, 1, 1], padding="SAME")
    print("convolution 2.")
    norm2 = tf.nn.local_response_normalization(conv2,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    print("normalization 2.")
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    print("max pool 2.")
                                        
    print("")
    flat = flat_layer(pool2)
    print("flat.")
    fc_relu1 = fc_relu(flat)
    print("fully connected relu 1.")
    fc_relu2 = fc_relu(fc_relu1)
    print("fully connected relu 2.")
    
    # Stop the threads
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)
    sess.close()

convolution 1.
max pool 1.
normalization 1.

convolution 2.
normalization 2.
max pool 2.

flat.
fully connected relu 1.
fully connected relu 2.
